In [1]:
import os 

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\mahik\\Documents\\Git_serious\\Fine-tuning-text-summarizer---end-to-end-deployment'

In [ ]:
# entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    base_model: Path
    # params
    per_device_train_batch_size : int
    per_device_eval_batch_size : int  
    num_train_epochs : int             
    learning_rate : float       
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int
    gradient_checkpointing : bool
    fp16 :bool              
    report_to : str  


configuration manager : 

In [ ]:
from src.Text_summarizer.constants import *
from src.Text_summarizer.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        # root directory
        create_directories([config.root_dir])

        # model trainer config
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            base_model = config.base_model,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            num_train_epochs = params.num_train_epochs,
            learning_rate = params.learning_rate,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            gradient_checkpointing = params.gradient_checkpointing,
            fp16 = params.fp16,
            report_to = params.report_to
        )
        return model_trainer_config



Components :

In [ ]:
from transformers import pipeline , set_seed
import os
from datasets import load_dataset , load_from_disk
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments , Trainer
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 

    def train(self):    
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.base_model)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.base_model).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        # loading data 
        dataset_samsum = load_from_disk(self.config.data_path)
        # trainer
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=500,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            gradient_checkpointing=self.config.gradient_checkpointing,
            fp16=self.config.fp16,
            report_to=self.config.report_to
        )
        
        trainer_args.data_collator = seq2seq_data_collator

        # use only 1000 samples for training 

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum["train"].shuffle().select(range(1000)),
            eval_dataset=dataset_samsum["validation"])

        trainer.train()
        # SAVE MODEL
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"Trained-pegasus-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"new-tokenizer"))


main.py : 

In [ ]:
try :
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_train = ModelTrainer(config=model_trainer_config)
    model_train.train()
except Exception as e:
    raise e